Using the Apache Beam interactive runner with JupyterLab notebooks lets you iteratively develop pipelines, inspect your pipeline graph, and parse individual PCollections in a read-eval-print-loop (REPL) workflow.

Apache Beam is installed on your notebook instance.

In [1]:
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

If your notebook uses other Google services:

In [2]:
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
import google.auth

## Interactivity Options
data capture duration to 60 seconds. If you want to iterate faster, set it to a lower duration, for example ‘10s'.

In [3]:
ib.options.recording_duration = '60s'

## Initialize the pipeline using an InteractiveRunner object

In [4]:
# Setting up the Apache Beam pipeline options.
options = pipeline_options.PipelineOptions()

# Set the pipeline mode to stream the data from Pub/Sub.
options.view_as(pipeline_options.StandardOptions).streaming = True

# Sets the project to the default project in your current Google Cloud environment.
# The project will be used for creating a subscription to the Pub/Sub topic.
_, options.view_as(GoogleCloudOptions).project = google.auth.default()

# The Google Cloud PubSub topic for this example.
topic = "projects/pubsub-public-data/topics/shakespeare-kinglear"

p = beam.Pipeline(InteractiveRunner(), options=options)

## Reading
an Apache Beam pipeline that creates a subscription to the given Pub/Sub topic and reads from the subscription

In [5]:
words = p | "read" >> beam.io.ReadFromPubSub(topic=topic)

**p** counts the words by windows from the source. It creates fixed windowing with each window being 10 seconds in duration.

In [6]:
windowed_words = (words
   | "window" >> beam.WindowInto(beam.window.FixedWindows(10)))
windowed_word_counts = (windowed_words
   | "count" >> beam.combiners.Count.PerElement())

## Visualizing the data
`show()` method visualizes the resulting PCollection in the notebook.

In [7]:
ib.show(windowed_word_counts, include_window_info=True)

<IPython.core.display.Javascript object>

apply multiple filters to your visualizations. The following visualization allows you to filter by label and axis:

In [8]:
ib.show(windowed_word_counts, include_window_info=True, visualize_data=True)

<IPython.core.display.Javascript object>

### output in a Pandas DataFrame
first converts the words to lowercase and then computes the frequency of each word.

In [9]:
windowed_lower_word_counts = (windowed_words
   | "to lower case" >> beam.Map(lambda word: word.lower())
   | "count lowered" >> beam.combiners.Count.PerElement())
ib.collect(windowed_lower_word_counts, include_window_info=True)

,0,1,event_time,windows,pane_info
0,b'not',2,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
1,b'to',1,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
2,b'the',2,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
3,b'advantages',1,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
4,b'of',1,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
5,b'france',1,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
6,b'o',1,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
7,b'heavens',1,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
8,b'that',1,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
9,b'this',1,1693749509999999,"[[1693749500.0, 1693749510.0)]","PaneInfo(first: True, last: False, timing: ON_..."
